In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import ElasticNetCV
from os import listdir

In [2]:
#Import metadata
long_data=pd.read_csv("/scratch/ckelsey4/Cayo_meth/long_data_adjusted.txt", sep="\t")
long_data=long_data[long_data['n'] > 1]
long_data=long_data.sort_values(by=['lid_pid'])

In [2]:
#Define natural sorting function
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

In [6]:
#Import coverage data per chromosome
path="/scratch/ckelsey4/Cayo_meth/dnam_clock/"

cov_files=[f for f in listdir(path)
        if "cov" in f]

cov_files=sorted(cov_files, key=natural_key)        

#cov_files=[path + f for f in cov_files]

cov_full=[]

for file in cov_files:
        file=path + file
        dd=pd.read_csv(file, sep=" ")
        cov_full.append(dd)



In [3]:
#Import count data
cpgs_m=pd.read_csv("/scratch/ckelsey4/Cayo_meth/dnam_clock/cpg_m.22.txt", sep=" ")

AttributeError: 'str' object has no attribute 'shape'